# Deep Learning to Identify Traffic Signs

Implemenation of deep neural network to identify traffic signs according to [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

A paper titled ["Traffic Sign Recognition with Multi-Scale Convolutional Networks"](https://ieeexplore.ieee.org/abstract/document/6033589) was used as a baseline for the model and the [LeNet-5 architecture](https://ieeexplore.ieee.org/abstract/document/726791) by the same author was implemented here to train the model.

---
## Load The Data

In [ ]:
# Load pickled data
import pickle

training_file = './traffic-signs-data/train.p'
validation_file = './traffic-signs-data/valid.p'
testing_file = './traffic-signs-data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

---

## Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image.

### Basic Summary of the Data Set

In [ ]:
num_train, img_height, img_width, img_depth = train['features'].shape

n_train = num_train

n_validation = len(X_valid)

n_test = len(X_test)

image_shape = (img_height, img_width, img_depth)

y_train_set = set(y_train)
y_valid_set = set(y_valid)
y_test_set = set(y_test)
n_classes = len(y_train_set.union(y_valid_set).union(y_test_set))

print("Number of training examples =", n_train)
print("Number of validation examples =", n_validation)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Sample of each traffic sign

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

signs = dict.fromkeys(range(0, 43))
for k in signs:
    indices = np.where(y_train==k)[0] 
    if indices.any():
        signs[k] = X_train[indices.item(0)]
        continue
    indices = np.where(y_valid==k)[0] 
    if indices.any():
        signs[k] = X_valid[indices.item(0)]
        continue
    indices = np.where(y_test==k)[0] 
    if indices.any():
        signs[k] = X_test[indices.item(0)]

        
plt.figure(figsize=(20, 20))
for code, image in signs.items():
    plt.subplot(9, 5, code+1)
    plt.axis("off")
    plt.subplots_adjust(hspace=0.3)
    plt.title(str(code))
    plt.imshow(image)


### Preprocess the Data Set (Normalization and shuffling)

In [ ]:
from sklearn.utils import shuffle

# Normalize the data
X_train = ((X_train - 127.5) / 127.5)
X_valid = ((X_valid - 127.5) / 127.5)
X_test = ((X_test - 127.5) / 127.5)

X_train, y_train = shuffle(X_train, y_train)

### Model Architecture

In [ ]:
import tensorflow as tf
from tensorflow.contrib.layers import flatten

EPOCHS = 50
BATCH_SIZE = 200

def LeNet(x):
    mu = 0
    sigma = 0.1
    
    # Layer 1: Convolutional. Input = 32x32x3. Output = 28x28x6.
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 3, 6), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(6))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b

    # Activation.
    conv1 = tf.nn.relu(conv1)

    # Pooling. Input = 28x28x6. Output = 14x14x6.
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # Layer 2: Convolutional. Output = 10x10x16.
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 16), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(16))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    
    # Activation.
    conv2 = tf.nn.relu(conv2)

    # Pooling. Input = 10x10x16. Output = 5x5x16.
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # Flatten. Input = 5x5x16. Output = 400.
    fc0   = flatten(conv2)
    
    # Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(400, 120), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(120))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    
    # Activation.
    fc1    = tf.nn.relu(fc1)

    # Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(120, 84), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(84))
    fc2    = tf.matmul(fc1, fc2_W) + fc2_b
    
    # Activation.
    fc2    = tf.nn.relu(fc2)

    # Layer 5: Fully Connected. Input = 84. Output = 43.
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(84, 43), mean = mu, stddev = sigma))
    fc3_b  = tf.Variable(tf.zeros(43))
    logits = tf.matmul(fc2, fc3_W) + fc3_b
    
    return logits

# 
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, 43)

rate = 0.002

logits = LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss = tf.reduce_mean(cross_entropy)
#optimizer = tf.train.GradientDescentOptimizer(learning_rate = rate)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
prediction = tf.argmax(logits, 1)
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

### Train, Validate and Test the Model

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
            
        validation_accuracy = evaluate(X_valid, y_valid)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './lenet')
    print("Model saved")

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

### Testing of various image sets

#### Unperturbed Signs

In [ ]:
import glob
import cv2

images = []
for path in glob.glob('unperturbed_signs/*.png'):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_arr = np.asarray(image)
    images.append(image_arr)

images = np.asarray(images)

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    for i, img in enumerate(images):
        pred = sess.run(prediction, feed_dict={x: np.array([img])})
        plt.subplot(5,2,i+1)
        plt.subplots_adjust(hspace=0.8)
        plt.axis("off")
        plt.title("Prediction: " + str(pred[0]))
        plt.imshow(img)  

    # Preprocess
    X_test2 = ((images - 127.5) / 127.5)
    y_test2 = np.array([2, 28, 32, 11, 17, 40, 23, 0, 4, 14])

    ### Calculate the accuracy for these 8 new images.
    test_accuracy = evaluate(X_test2, y_test2)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

#### Gaussian Noise

In [ ]:
import glob
import cv2

images = []
for path in glob.glob('gauss_signs/*.png'):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_arr = np.asarray(image)
    images.append(image_arr)

images = np.asarray(images)

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    for i, img in enumerate(images):
        pred = sess.run(prediction, feed_dict={x: np.array([img])})
        plt.subplot(5,2,i+1)
        plt.subplots_adjust(hspace=0.8)
        plt.axis("off")
        plt.title("Prediction: " + str(pred[0]))
        plt.imshow(img)  

    # Preprocess
    X_test2 = ((images - 127.5) / 127.5)
    y_test2 = np.array([2, 28, 32, 11, 17, 40, 23, 0, 4, 14])

    ### Calculate the accuracy for these 8 new images.
    test_accuracy = evaluate(X_test2, y_test2)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

#### Poisson Noise

In [ ]:
import glob
import cv2

images = []
for path in glob.glob('poisson_signs/*.png'):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_arr = np.asarray(image)
    images.append(image_arr)

images = np.asarray(images)

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    for i, img in enumerate(images):
        pred = sess.run(prediction, feed_dict={x: np.array([img])})
        plt.subplot(5,2,i+1)
        plt.subplots_adjust(hspace=0.8)
        plt.axis("off")
        plt.title("Prediction: " + str(pred[0]))
        plt.imshow(img)  

    # Preprocess
    X_test2 = ((images - 127.5) / 127.5)
    y_test2 = np.array([2, 28, 32, 11, 17, 40, 23, 0, 4, 14])

    ### Calculate the accuracy for these 8 new images.
    test_accuracy = evaluate(X_test2, y_test2)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

#### Salt and Pepper Noise

In [ ]:
import glob
import cv2

images = []
for path in glob.glob('salt_and_pep_signs/*.png'):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_arr = np.asarray(image)
    images.append(image_arr)

images = np.asarray(images)

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    for i, img in enumerate(images):
        pred = sess.run(prediction, feed_dict={x: np.array([img])})
        plt.subplot(5,2,i+1)
        plt.subplots_adjust(hspace=0.8)
        plt.axis("off")
        plt.title("Prediction: " + str(pred[0]))
        plt.imshow(img)  

    # Preprocess
    X_test2 = ((images - 127.5) / 127.5)
    y_test2 = np.array([2, 28, 32, 11, 17, 40, 23, 0, 4, 14])

    ### Calculate the accuracy for these 8 new images.
    test_accuracy = evaluate(X_test2, y_test2)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

#### Speckle Noise

In [ ]:
import glob
import cv2

images = []
for path in glob.glob('speckle_signs/*.png'):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_arr = np.asarray(image)
    images.append(image_arr)

images = np.asarray(images)

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    for i, img in enumerate(images):
        pred = sess.run(prediction, feed_dict={x: np.array([img])})
        plt.subplot(5,2,i+1)
        plt.subplots_adjust(hspace=0.8)
        plt.axis("off")
        plt.title("Prediction: " + str(pred[0]))
        plt.imshow(img)  

    # Preprocess
    X_test2 = ((images - 127.5) / 127.5)
    y_test2 = np.array([2, 28, 32, 11, 17, 40, 23, 0, 4, 14])

    ### Calculate the accuracy for these 8 new images.
    test_accuracy = evaluate(X_test2, y_test2)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

#### Random Noise

In [ ]:
import glob
import cv2

images = []
for path in glob.glob('random_noise_signs/*.png'):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_arr = np.asarray(image)
    images.append(image_arr)

images = np.asarray(images)

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    for i, img in enumerate(images):
        pred = sess.run(prediction, feed_dict={x: np.array([img])})
        plt.subplot(5,2,i+1)
        plt.subplots_adjust(hspace=0.8)
        plt.axis("off")
        plt.title("Prediction: " + str(pred[0]))
        plt.imshow(img)  

    # Preprocess
    X_test2 = ((images - 127.5) / 127.5)
    y_test2 = np.array([2, 28, 32, 11, 17, 40, 23, 0, 4, 14])

    ### Calculate the accuracy for these 8 new images.
    test_accuracy = evaluate(X_test2, y_test2)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

#### FGSM Adversarial Images

In [ ]:
import glob
import cv2

images = []
for path in glob.glob('adversarial_signs/*.png'):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_arr = np.asarray(image)
    images.append(image_arr)

images = np.asarray(images)

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    for i, img in enumerate(images):
        pred = sess.run(prediction, feed_dict={x: np.array([img])})
        plt.subplot(5,2,i+1)
        plt.subplots_adjust(hspace=0.8)
        plt.axis("off")
        plt.title("Prediction: " + str(pred[0]))
        plt.imshow(img)  

    # Preprocess
    X_test2 = ((images - 127.5) / 127.5)
    y_test2 = np.array([2, 28, 32, 11, 17, 40, 23, 0, 4, 14])

    ### Calculate the accuracy for these 8 new images.
    test_accuracy = evaluate(X_test2, y_test2)
    print("Test Accuracy = {:.3f}".format(test_accuracy))